In [1]:
import keras
import pandas as pd
import numpy as np
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
# load data_set
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

In [3]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [12]:
## Split the data in train validation and test 
train,test = train_test_split(data,test_size=0.25,random_state=55)
train_x = train.drop(['quality'],axis=1).values
train_y = train['quality'].values.ravel()

test_x = test.drop(['quality'],axis=1).values
test_y = test['quality'].values.ravel()

#Validation data
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.20,random_state=55)

signature = infer_signature(train_x,train_y)

In [25]:
# ANN model
def train_model(params,epochs,train_x,train_y,val_x,val_y,test_x,test_y):
    
    # Define model Architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)


    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    #Compile the model
    model.compile(optimizer = keras.optimizers.SGD(
        learning_rate = params['lr'],
        momentum = params['momentum']),
        loss = 'mean_squared_error',
        metrics = [keras.metrics.RootMeanSquaredError()]
        )

    #Train the model
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(val_x,val_y),
        epochs = 3,
        batch_size = 64)

        #Evaluate the model
        eval_result = model.evaluate(val_x,val_y,batch_size = 64)
        eval_rmse = eval_result[1]

        #Log the prameters
        mlflow.log_params(params)
        mlflow.log_metric('Eval_rmse',eval_rmse)

        #Log the model
        mlflow.tensorflow.log_model(model,"Model",signature=signature)

        return {'loss':eval_rmse, 'status':STATUS_OK, 'model':model}

In [27]:
def objective(params):
    # MLflow will track the parameters and resutls for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        val_x=val_x,
        val_y=val_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [16]:
space = {
    'lr':hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
    'momentum':hp.uniform('momentum',0.0,1.0)
}

In [28]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    #Conduct the hyperparameters search using hyperopt
    trials = Trials()
    best = fmin(
        fn = objective,
        space= space,
        algo = tpe.suggest,
        max_evals = 4,
        trials = trials
    )

    #Fetch details of the best runs
    best_run = sorted(trials.results,key = lambda x:x['loss'])[0]

    #log the best parameters
    mlflow.log_params(best)
    mlflow.log_metric("Eval_rmse",best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'],'Best model',signature=signature)


    #Print out the best parameters and corrosponding loss
    print(f"Best Parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 16s 363ms/step - loss: 33.5134 - root_mean_squared_error: 5.7891
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.6024 - root_mean_squared_error: 2.5987    
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.2191 - root_mean_squared_error: 2.3258 - val_loss: 0.6255 - val_root_mean_squared_error: 0.7909

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.5601 - root_mean_squared_error: 0.7484
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5788 - root_mean_squared_error: 0.7606 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5784 - root_mean_squared_error: 0.7603 - val_loss: 0.5591 - val_root_mean_squared_error: 0.7477

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.5337 - root_mean_squared_error: 0.7305
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5325 - root_mean_squared_error: 0.7297 - va

In [29]:
#Validate the model

from mlflow.models import validate_serving_input

model_uri = 'runs:/535cdf41ef1244928bd3f9a5b509dd9a/Model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

d:\ML projects\Learning\MLFlow\MlFlowStarter\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[5.974147 ],
       [5.7163086],
       [5.6488905],
       ...,
       [6.4851284],
       [5.4002023],
       [5.296697 ]], dtype=float32)

In [30]:
#Register the model
mlflow.register_model(model_uri,"Wine-quality")

Successfully registered model 'Wine-quality'.
Created version '1' of model 'Wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1736687403018, current_stage='None', description=None, last_updated_timestamp=1736687403018, name='Wine-quality', run_id='535cdf41ef1244928bd3f9a5b509dd9a', run_link=None, source='file:///d:/ML%20projects/Learning/MLFlow/MlFlowStarter/WineQuality/mlruns/983187369760308651/535cdf41ef1244928bd3f9a5b509dd9a/artifacts/Model', status='READY', status_message=None, tags={}, user_id=None, version=1>